In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [ ]:
diabetes = pd.read_csv('data/diabetes.csv')

In [ ]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = (diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min())

In [ ]:
drop_indices = np.random.choice(diabetes.index, 368, replace=False)
diabetes = diabetes.drop(drop_indices)

In [ ]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome',axis=1), diabetes['Outcome'], test_size=0.2)

In [ ]:
X_train[X_train == 0] = 0.000001
X_test[X_test == 0] = 0.000001

In [ ]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [ ]:
y_train = [int(x) for x in y_train]
y_test =  [int(x) for x in y_test]

In [ ]:
"""
with open('results/version4/train_data.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version4/test_data.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)
"""

In [2]:
with open('results/version4/train_data.json') as json_file:
    X_train = json.load(json_file)[0]

with open('results/version4/train_data.json') as json_file:
    y_train = json.load(json_file)[1]
    
with open('results/version4/test_data.json') as json_file:
    X_test = json.load(json_file)[0]

with open('results/version4/test_data.json') as json_file:
    y_test = json.load(json_file)[1]

## Test

In [ ]:
weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                           threshold=0.7,
                           lrParameter=0.1, 
                           n_epochs=20, 
                           trainingBias=False,
                           epoch_results=True,
                           phaseEstrategyOperator='original',
                           trainingApproaches={'hsgs','phase-encoding'})

In [ ]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.7,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=10,
                                   bias=False, 
                                   phaseEstrategyOperator='original',
                                   testingApproaches={'hsgs', 'phase-encoding'})

In [ ]:
print('phase avg_error', results[0])
print('phase weights', results[2])

print('\HSGS avg_error', results[3])
print('HSGS weights', results[5])

#print('phase neuron_outputs: ', results[1])
#print('HSGS neuron_outputs: ', results[4])
#print('targets: ', results[6])
#print('examples: ', results[7])

## Search

In [3]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [5]:
# search space

initial_weights = [
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 #[0.9091283748867313, 0.13316944575925016],
 #[0.5234125806737658, 0.7504098591020348],
 #[0.6690132408839138, 0.4677528597449807],
 #[0.20484909029779508, 0.49076588909107044],
 #[0.372384689385059, 0.4774011548515884]
]

threshold_space = [0.8, 0.2, 0.5]
lr_space = [0.01, 0.05, 0.1]
bias_space = [False, True]
strategy_space = ['radius', 'angleradius', 'original', 'angle']

In [6]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=w,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=20, 
                                               trainingBias=bias,
                                               epoch_results=True,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs','phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=5,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'hsgs', 'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)

                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[3])
                    output['neuron_outputs'].append(results[4])
                    output['trained_weights'].append(results[5])
                    output['initial_weights'].append(w)
                    print('\n')


lrParameter:  0.01
threshold:  0.8
trainingBias:  False
phaseEstrategyOperator:  radius

erro HSGS 196
weights HSGS [0.8858826929411794, 1.5906030350753777, 1.6030328668852456, 0.8792934892929308, 0.6704737232387749, 1.394038768137109, 0.7926558497011099, 0.831000219999999]
erro phase encoding 124
weights phase encoding [-0.4999073234202995, -0.2944582948301276, -0.19377274934910557, 0.039960826847984965, 0.5, 0.5, 0.5, 0.5]

erro HSGS 196
weights HSGS [1.271765385882354, 2.681206070150756, 2.7060657337704943, 1.258586978585856, 0.8409474464775497, 2.2880775362742156, 1.0853116994022196, 1.1620004399999957]
erro phase encoding 124
weights phase encoding [-1.4998146468405995, -1.0889165896602537, -0.8875454986982111, -0.4200783463040301, 0.5, 0.5, 0.5, 0.5]

erro HSGS 196
weights HSGS [1.6576480788235257, 3.771809105226133, 3.8090986006557452, 1.6378804678787793, 1.011421169716323, 3.182116304411324, 1.3779675491033283, 1.4930006599999919]
erro phase encoding 124
weights phase encoding 

KeyboardInterrupt: 

In [7]:
experiment_data = pd.DataFrame(output)
experiment_data

,model,phase_strategy,bias,threshold,lr,avg_error,trained_weights,initial_weights,neuron_outputs
0,phase-encoding,radius,False,0.8,0.01,0.3375,"[-19.155118567723576, -15.139033340397166, -13...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,HSGS,radius,False,0.8,0.01,0.6750,"[8.217653858823546, 22.312060701507576, 22.560...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,phase-encoding,radius,True,0.8,0.01,0.3250,"[-19.163891598611208, -15.160235356664918, -13...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,HSGS,radius,True,0.8,0.01,0.6750,"[8.217653858823546, 22.312060701507576, 22.560...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,phase-encoding,radius,False,0.8,0.05,0.3250,"[-94.41081270762986, -74.22464631123657, -65.0...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,HSGS,radius,False,0.8,0.05,0.6750,"[39.088269294118824, 109.56030350753747, 110.8...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,phase-encoding,radius,True,0.8,0.05,0.3250,"[-95.71262711069325, -75.40744418911846, -66.1...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,HSGS,radius,True,0.8,0.05,0.6750,"[39.088269294118824, 109.56030350753747, 110.8...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,phase-encoding,radius,False,0.8,0.10,0.3250,"[-188.35378928517136, -148.56856594923372, -12...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,HSGS,radius,False,0.8,0.10,0.6750,"[77.67653858823769, 218.6206070150749, 221.106...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [8]:
experiment_data.to_csv('results/experiment_diabetes_radius2.csv')